# U-NET page-segmentation 학습이미지 데이터 증강
- tf api의 image 클래스활용
- 이미지 데이터 증강기법(https://www.tensorflow.org/addons/tutorials/image_ops?hl=ko)
  - tfa.image.rotate : 단순 회전(2D)
  - tfa.image.transform : 벡터기반으로 회전(3D)
  - tfa.Random HSV in YIQ : 색상스케일, 색조, 채도등 무작위 지정 
  - tf.image.flip_left_right : 상하 또는 좌우 뒤집기
  - tf.image.rgb_to_grayscale : 그레이 스케일로 변환
  - tf.image.adjust_saturation : 채도 조정
  - tf.image.adjust_brightness : 밝기 변환 
  - 위의 데이터는 수동으로 조정한다.
  - tf.image.stateless_random_brightness 등을 활용하면 랜덤으로 선택되어 변환
- 학습데이터의 세그먼테이션 라벨 이미지도 같은 랜덤시드로 증강해주어야 짝이 맞게 된다.


In [90]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd
import glob

import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Nadam
import tensorflow as tf
import tensorflow.python.keras.backend as k

from skimage.transform import resize

In [91]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 1


#데이터 경로 지정  
TRAIN_PATH = './page_data/train/'
TEST_PATH = './page_data/test/'

# UserWarning을 무시하는 설정(없음 OK)
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

#이미지 파일명을 리스트 형식으로 리턴  
train_imgs = glob.glob(TRAIN_PATH+'org/*.jpg')
train_masks = glob.glob(TRAIN_PATH+'seg/*.jpg')
test_imgs = glob.glob(TEST_PATH+'org/*.jpg')
test_masks = glob.glob(TEST_PATH+'seg/*.jpg')


#리스트 길이 리턴  
num_of_train_imgs = len(train_imgs)
num_of_train_masks = len(train_masks)
if num_of_train_imgs != num_of_train_masks:
    print('invalid datasets, please check train data')
    
#각이미지를 배열로 리턴
#image 
X_train = np.zeros((num_of_train_imgs, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
# mask
Y_train = np.zeros((num_of_train_masks, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()


Getting and resizing train images and masks ... 


In [93]:
for n in range(num_of_train_imgs):
    img = imread(train_imgs[n],as_gray=True)
    img = (img * 255).astype(np.uint8)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    
    mask = imread(train_masks[n],as_gray=True)
    mask_resized = resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    Y_train[n] = mask_resized.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    
# 테스트 이미지를 배열로 리턴
X_test = np.zeros((len(test_imgs), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_masks), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
print('Getting test images ... ')
sys.stdout.flush()

for n in range(len(test_imgs)):
    img = imread(test_imgs[n],as_gray=True)
    img = (img * 255).astype(np.uint8)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    
    mask = imread(test_masks[n],as_gray=True)
    mask_resized = resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    Y_test[n] = mask_resized.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

print('Preparing is Done!')


Getting test images ... 
Preparing is Done!


In [ ]:
# 데이터 증강 해볼것이다



In [32]:
# U-Net model 

inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255.) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = BatchNormalization() (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same') (p1)
c2 = LeakyReLU()(c2)
c2 = BatchNormalization() (c2)
c2 = Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same') (c2)
c2 = LeakyReLU()(c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same') (p2)
c3 = LeakyReLU()(c3)
c3 = BatchNormalization() (c3)
c3 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same') (c3)
c3 = LeakyReLU()(c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same') (p3)
c4 = LeakyReLU()(c4)
c4 = BatchNormalization() (c4)
c4 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same') (c4)
c4 = LeakyReLU()(c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = BatchNormalization() (c5)
c5 = Conv2D(256, (3, 3), activation='elu',kernel_initializer='he_normal', padding='same') (c5)

#===========================================

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same') (u6)
c6 = LeakyReLU()(c6)
c6 = BatchNormalization() (c6)
c6 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same') (c6)
c6 = LeakyReLU()(c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same') (u7)
c7 = LeakyReLU()(c7)
c7 = BatchNormalization() (c7)
c7 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same') (c7)
c7 = LeakyReLU()(c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same') (u8)
c8 = LeakyReLU()(c8)
c8 = BatchNormalization() (c8)
c8 = Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same') (c8)
c8 = LeakyReLU()(c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = BatchNormalization(momentum=0.9) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])

model.compile(optimizer=Nadam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['binary_accuracy','acc'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 lambda_1 (Lambda)              (None, 512, 512, 1)  0           ['input_2[0][0]']                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 512, 512, 16  160         ['lambda_1[0][0]']               
                                )                                                                 
                                                                                            

In [ ]:
# 모델 훈련
checkpointer = ModelCheckpoint('model-page-004da.h5', verbose=1, save_best_only=True)
reduceLr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=7)


results = model.fit(X_train, 
                    Y_train, 
                    validation_split=0.1, 
                    batch_size=3, 
                    epochs=50,
                    callbacks=[reduceLr, checkpointer])

In [94]:
model = load_model('model-page-004da.h5')

In [ ]:
preds_test = model.predict(X_test, verbose=1)

#픽셀  평가 결과를 0.5의 임계치로 0 or 255로 변환(평가 결과를 바이너리 이미지로 변환).
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# # 임의 결과로 하나를 리턴 받아 표시 
ix = random.randint(0, len(preds_test_t))
ix = 0
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.title('org')
plt.imshow(X_test[ix].reshape(IMG_HEIGHT, IMG_WIDTH), cmap='gray')

plt.subplot(1,3,2)
plt.title('seg')
plt.imshow(Y_test[ix].reshape(IMG_HEIGHT, IMG_WIDTH), cmap='gray')

plt.subplot(1,3,3)
plt.title('pred_seg')
plt.imshow(np.squeeze(preds_test_t[ix].reshape(IMG_HEIGHT, IMG_WIDTH)), cmap='gray')

In [ ]:
preds_test = model.predict(X_train, verbose=1)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

ix = 11
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title('org')
plt.imshow(X_train[ix].reshape(IMG_HEIGHT, IMG_WIDTH), cmap='gray')

plt.subplot(1, 3, 2)
plt.title('seg')
plt.imshow(Y_train[ix].reshape(IMG_HEIGHT, IMG_WIDTH), cmap='gray')

plt.subplot(1, 3, 3)
plt.title('predicted_seg')
plt.imshow(np.squeeze(preds_test_t[ix].reshape(IMG_HEIGHT, IMG_WIDTH)), cmap='gray')